In [2]:
import os
import subprocess

In [3]:
def missing_gt(samples, processed):
    return [i for i in samples if not i in processed]


In [4]:
def harvest_gt(gt_folder, maxchr=30):
    F1 = os.listdir(gt_folder)
    samples = [i.split(".")[0] for i in F1 if i.endswith("genotype")]
    folders1 = [i for i in F1 if i.endswith("DIR")]
    nothing = [i for i in samples if not i+".genotype_DIR" in folders1]
    
    sorted_files = []
    split_files = []
    flipped_files = []
    
    for folder in folders1:
        lf = os.path.join(gt_folder, folder)
        ldir = os.listdir(lf)
        sortedgt = [i for i in ldir if i.endswith(".sorted")]
        chr_1s = [i for i in ldir if i.endswith(".chr.{}".format(maxchr))]
        if len(chr_1s)>=1:
            split_files.append(chr_1s[0].split(".")[0])
            if len(chr_1s)>1:
                flipped_files.append(chr_1s[0].split(".")[0])
        if len(sortedgt)>=1:
            sorted_files.append(sortedgt[0].split(".")[0]) 
    
    missing_sort = missing_gt(samples, sorted_files)
    missing_split = missing_gt(samples, split_files)
    missing_flip = missing_gt(samples, flipped_files)

    jobstate_dict = {"not_sorted":missing_sort,
                     "not_split":missing_split,
                     "not_flipped":missing_flip}
    return  nothing, samples,jobstate_dict

In [5]:
def harvest_not_completely_done(samples, chromosomes, windowsize, gtdir, tgdir):
    F2 = [i.split(".")[0] for i in os.listdir(tgdir)]
    
    # create dict for the ones that dont have any data at all
    completely_no_tig = missing_gt(samples, F2)
    fresh_not_done_all = {}
    for folder in completely_no_tig:
        fresh_not_done_all.setdefault(folder,{})
        #for windowsize in windowsizes:
        fresh_not_done_all[folder].setdefault(str(windowsize), [])  #add indent here for retooling
        for chromosome in chromosomes: #add indent here for retooling
            fresh_not_done_all[folder][str(windowsize)].append(str(chromosome)) #add indent here for retooling
    
    # create dictionary with all files that are supposed to be there.
    supp_all = {}
    for folder in F2:
        supp_all.setdefault(folder,{})
        #for windowsize in windowsizes:
        supp_all[folder].setdefault(str(windowsize), []) #add indent here for retooling
        for chromosome in chromosomes:  #add indent here for retooling

            #supp_sample = "{}.genotype.{}.rough_COs_windowsize{}.refined.recomb.txt".format(folder, chromosome, windowsize)
            #supp_all[folder][str(windowsize)].append(supp_sample)
            supp_all[folder][str(windowsize)].append(str(chromosome))  #add indent here for retooling

    # create dictionary with all files that are there
    present_all = {}
    for folder in F2:
        #print(folder)
        fpF2 = os.path.join(tgdir,folder+".genotype")
        #print(fpF2)
        actually_there = [i for i in os.listdir(fpF2) if i.endswith("refined.recomb.txt")]
        #print(actually_there)
        for file in actually_there:
            split_file = file.split(".")
            if "flipped" in split_file:
                pass  #to be implemented later
            else:
                sample = split_file[0]
                chromosome = split_file[2] 
                windowsize = split_file[3].lstrip("rough_COs_windowsize")
                present_all.setdefault(sample,{}).setdefault(str(windowsize),[]).append(chromosome)
                #present_all.setdefault(sample,{}).setdefault(windowsize,[]).append(file)
    
    # create dictionary with files that are missing
    needed_to_be_done = {}
    for sample, item in supp_all.items():
        if not sample in present_all.keys():
            #print("sample is not present at all, redo {}".format(sample))
            needed_to_be_done[sample]=item
        else:
            #print("processing sample {}".format(sample))
            for windowsize, item2 in item.items():
                #print("    processing windowsize {}".format(windowsize))
                if not windowsize in present_all[sample].keys():
                    #print("        windowsize {} not present, adding whole".format(windowsize))
                    needed_to_be_done.setdefault(sample, {})[windowsize]=item2
                else:
                    for item3 in item2:
                        if not item3 in dict.fromkeys(present_all[sample][windowsize]):
                            needed_to_be_done.setdefault(sample, {}).setdefault(windowsize, []).append(item3)
    
    return fresh_not_done_all, needed_to_be_done

In [6]:
def create_sort_job(sample_name, target_dir):
    inputfile  = os.path.join(target_dir,
                              sample_name+".genotype")
    outputfile = os.path.join(target_dir,
                              sample_name+".genotype_DIR",
                              sample_name+".genotype.sorted")
    
    jobstring  = "sort -k 1,1n -k 2,2n {inputfile} > {outputfile}".format(inputfile=inputfile, outputfile=outputfile)
    return jobstring

def create_split_job(sample_name, target_dir, index, maxchr=30):
    f = sample_name+".genotype"
    input_gt_file = os.path.join(target_dir,
                                 f+"_DIR",
                                 f+".sorted")
    outpath = os.path.join(target_dir,f+"_DIR")
    jobstring = "python scripts/vcf2individualGenotypes/split_by_chrom.py -i {input_gt_file}  -o {outpath}  -s {index}  -m {maxchr} -d {f}".format(input_gt_file=input_gt_file, outpath=outpath,index=index,maxchr=maxchr,f=f)
    return jobstring

def create_flip_job(sample_name, target_dir):
    pass


def create_caller(chromosome, sample_name, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    cmdCaller = "scripts/individualGenotypes2breaks/base_caller.jar"
    in_1 = os.path.join(gt_dir, f+"_DIR", f+".chr.{chr}".format(chr=str(chromosome)))
    caller_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_allele_co.txt".format(chromosome=chromosome, winsize=winsize) )
    jobstring = "java -Xmx4G -jar  {cmdCaller}  -r {in_1} -o {caller_out} -n bi ".format(cmdCaller=cmdCaller,
                                                                                           in_1=in_1,
                                                                                           caller_out=caller_out)
    return jobstring
    
def create_estimate(chromosome, sample_name, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    in_1  = os.path.join(gt_dir, f+"_DIR", f+".chr.{chr}".format(chr=chromosome))
    cmdEstimate  = "scripts/individualGenotypes2breaks/allele_freq_estimator.jar"
    out_estimate = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_frequencies_for_hmm.txt".format(chromosome=chromosome, winsize=winsize) )
    jobstring =  "java -Xmx4G -jar {cmdEstimate}  -r {in_1} -o {out_estimate} -n bi -w {winsize} ".format(cmdEstimate=cmdEstimate,
                                                                                                               in_1=in_1,
                                                                                                               winsize=winsize,
                                                                                                               out_estimate=out_estimate)
    return jobstring

def create_beta(chromosome, sample_name, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    in_1  = os.path.join(gt_dir, f+"_DIR", f+".chr.{chr}".format(chr=chromosome))
    cmdBeta = "scripts/individualGenotypes2breaks/beta_mixture_model.R"
    # beta_in is estimate_out
    beta_in = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_frequencies_for_hmm.txt".format(chromosome=chromosome, winsize=winsize) )
    beta_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_bmm.intersections.txt".format(chromosome=chromosome, winsize=winsize) )
    jobstring = "Rscript --vanilla {cmdBeta} {beta_in} {beta_out} ".format(cmdBeta=cmdBeta, beta_in=beta_in, beta_out=beta_out)
    return jobstring

def create_prep_prob(chromosome, sample_name,genomesize, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    in_1  = os.path.join(gt_dir, f+"_DIR", f+".chr.{chr}".format(chr=chromosome))
    cmdPrep = "scripts/individualGenotypes2breaks/prep_prob.pl"
    caller_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_allele_co.txt".format(chromosome=chromosome, winsize=winsize) )
    prep_prob_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_file_for_probabilities.txt".format(chromosome=chromosome, winsize=winsize) )
    jobstring = "perl {cmdPrep} -s {sample_name} -m {in_1} -b {caller_out} -c {genomesize} -o {prep_prob_out} ".format(cmdPrep=cmdPrep, sample_name=sample_name, in_1=in_1, caller_out=caller_out, genomesize=genomesize, prep_prob_out=prep_prob_out)

    return jobstring

def create_hmm_prob(chromosome, sample_name,genomesize, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    cmdHmm = "scripts/individualGenotypes2breaks/hmm_prob.pl"
    outshort =  os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}".format(chromosome=chromosome, winsize=winsize) )
    estimate_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_frequencies_for_hmm.txt".format(chromosome=chromosome, winsize=winsize) )
    prep_prob_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_file_for_probabilities.txt".format(chromosome=chromosome, winsize=winsize) )
    beta_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_bmm.intersections.txt".format(chromosome=chromosome, winsize=winsize) )
    jobstring = "perl {cmdHmm}  -s {estimate_out} -p {prep_prob_out} -o {outshort} -a {beta_out} -c {genomesize} ".format(cmdHmm=cmdHmm, estimate_out=estimate_out, prep_prob_out=prep_prob_out, outshort=outshort, beta_out=beta_out, genomesize=genomesize)
    return jobstring


# os.path.join(config["TIGER_WD"], "{study}" + config["TIGER_SUFFIX"], "{file}", "{file2}.{chr}.windowsize{winsize,[0-9]+}hmm_model")
def create_hmm(chromosome, sample_name, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    cmdHmm = "scripts/individualGenotypes2breaks/hmm_play.jar"
    caller_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_allele_co.txt".format(chromosome=chromosome, winsize=winsize) )
    hmm_prep_out = os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}hmm_model".format(chromosome=chromosome, winsize=winsize) )
    hmm_out =os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_hmm.txt".format(chromosome=chromosome, winsize=winsize) )
    jobstring = "java -jar {cmdHmm} -r {caller_out} -o {hmm_out} -t bi -z {hmm_prep_out} ".format(cmdHmm=cmdHmm,caller_out=caller_out,hmm_out=hmm_out,hmm_prep_out=hmm_prep_out)

    return jobstring

def create_prep_co(chromosome, sample_name,genomesize, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    in_1 = os.path.join(gt_dir, f+"_DIR", f+".chr.{chr}".format(chr=chromosome))
    cmdPrepCo ="scripts/individualGenotypes2breaks/prepare_break.pl"
    hmm_out =os.path.join(target_dir, f,f+".{chromosome}.windowsize{winsize}_hmm.txt".format(chromosome=chromosome, winsize=winsize) )
    prep_co_total = os.path.join(target_dir, f,f+".{chromosome}.rough_COs_windowsize{winsize}.txt".format(chromosome=chromosome, winsize=winsize))
    jobstring = "perl {cmdPrepCo} -s {sample_name} -m {in_1} -b {hmm_out} -c  {genomesize} -o {prep_co_total} ".format(cmdPrepCo=cmdPrepCo,prep_co_total=prep_co_total, sample_name=sample_name,in_1=in_1, genomesize=genomesize, hmm_out=hmm_out)

    return jobstring

def create_smooth_prep_co(chromosome, sample_name, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    cmdSmooth = "scripts/individualGenotypes2breaks/breaks_smoother.pl"
    prep_partial = os.path.join(target_dir, f,f+".{chromosome}.rough_COs_windowsize{winsize}.breaks.txt".format(chromosome=chromosome, winsize=winsize))
    smooth_out = os.path.join(target_dir, f,f+".{chromosome}.corrected.refined.breaks_windowsize{winsize}.txt".format(chromosome=chromosome, winsize=winsize) )
    jobstring = "perl {cmdSmooth} -b {prep_partial} -o {smooth_out} ".format(cmdSmooth=cmdSmooth, prep_partial=prep_partial, smooth_out=smooth_out)
    return jobstring

def create_refine(chromosome, sample_name, target_dir, gt_dir,winsize=200):
    f = sample_name+".genotype"
    in_1 = os.path.join(gt_dir, f+"_DIR", f+".chr.{chr}".format(chr=chromosome))
    cmdRefine = "scripts/individualGenotypes2breaks/refine_recombination_break.pl"
    prep_partial = os.path.join(target_dir, f,f+".{chromosome}.rough_COs_windowsize{winsize}.breaks.txt".format(chromosome=chromosome, winsize=winsize))
    jobstring = "perl {cmdRefine} {in_1} {prep_partial}".format(cmdRefine=cmdRefine, in_1=in_1, prep_partial=prep_partial)
    return jobstring


def create_tiger_job(chromosome, sample_name, target_dir, genomesize, gt_dir,winsize=200):
    jobstrings = []
    jobstrings.append(create_caller(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize))
    jobstrings.append(create_estimate(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize))
    jobstrings.append(create_beta(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize))
    jobstrings.append(create_prep_prob(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize, genomesize=genomesize))
    jobstrings.append(create_hmm_prob(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize, genomesize=genomesize))
    jobstrings.append(create_hmm(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize))
    jobstrings.append(create_prep_co(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize, genomesize=genomesize))
    jobstrings.append(create_smooth_prep_co(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize))
    jobstrings.append(create_refine(chromosome=chromosome, sample_name=sample_name, target_dir=target_dir, gt_dir=gt_dir,winsize=winsize))
    return jobstrings

In [7]:
def create_sbatch(joblines,name,conda_env="v3", hours="8",minutes="00", seconds="00", cores="2", user ="tilman.ronneburg@imbim.uu.se", proj="snic2017-7-53" ):
    sbatch_header = [ "#!/bin/bash -l",
                        "#SBATCH -A {proj}".format(proj=proj),
                        "#SBATCH -t {hours}:{minutes}:{seconds} ".format(hours=hours, minutes=minutes, seconds=seconds),
                        "#SBATCH -p core -n {cores}".format(cores=cores),
                        "#SBATCH -J {name}".format(name=name.rsplit(".",1)[0].rsplit("/",1)[1]),
                        "#SBATCH -o {name}_%j.out".format(name=name.rsplit(".",1)[0].rsplit("/",1)[1]),
                        "#SBATCH -e {name}_%j.error".format(name=name.rsplit(".",1)[0].rsplit("/",1)[1]),
                        "#SBATCH --mail-user {user}".format(user=user),
                        "#SBATCH --mail-type=FAIL",
                        "#SBATCH --get-user-env",
                        "##command underneath this##"
                       ]
    load_modules = ["source activate {env}".format(env=conda_env)]
    
    with open(name, "w") as handle:
        for line in sbatch_header:
            handle.write(line+"\n")
        for line in load_modules:
            handle.write(line+"\n")
        for line in joblines:
            handle.write(line+"\n")
    subprocess.call(["chmod", "766", str(name)])
    
def create_batchfiles(jobdict, scriptdir, prefix="batch_", execute=False):
    for sample, jobs in jobdict.items():
        print(sample)
        print(len(jobs))
        name = os.path.join(scriptdir, "{prefix}{sample}.job".format(prefix=prefix, sample=sample))
        #print(name)
        create_sbatch(joblines=jobs, name=name) # remember that this function has a lot of default options
        if execute==True:
            subprocess.call(["sbatch", name])

In [8]:
tgdir = "/home/tilman/storage3/process_stripes/stripes_optimise_F7/with.fam.f2.call2.TIGER_OUT/"
gtdir = "/home/tilman/storage3/process_stripes/stripes_optimise_F7/with.fam.f2.call2.Genotype/"
gsize =  "/home/tilman/storage3/stripes_optimise/aux/size.genome"
index =  "/home/tilman/storage3/stripes_optimise/aux/Index_fastq_contig_chr.txt"
winsize = 50
chromosomes = list(range(1,31))

In [9]:
nothing, samples, jstate = harvest_gt(gt_folder=gtdir, maxchr=30)
a, b = harvest_not_completely_done(samples=samples, chromosomes=chromosomes, windowsize=winsize, gtdir=gtdir, tgdir=tgdir)

In [13]:
#jstate
#a
#b

In [18]:
jobdict = {}

In [81]:
#jstate is empty, so we can ignore this thing
for sample in jstate["not_sorted"]:
    jobdict[sample] = []
    jobdict[sample].append("mkdir {}".format(os.path.join(gtdir,sample+".genotype_DIR")))
    jobdict[sample].append(create_sort_job(sample_name=sample, target_dir=gtdir))
    jobdict[sample].append(create_split_job(sample_name=sample, target_dir=gtdir, index=index))
    jobdict[sample].append("mkdir {}".format(os.path.join(tgdir,sample+".genotype")))
    for chromosome in chromosomes:
        tiger_string = " && ".join(create_tiger_job(chromosome=chromosome,sample_name=sample, target_dir=tgdir,genomesize=gsize, gt_dir=gtdir,winsize=winsize))
        jobdict[sample].append(tiger_string)
        
for sample in jstate["not_split"]:
    if not sample in jobdict.keys():
        jobdict[sample] = []
        jobdict[sample].append(create_split_job(sample_name=sample, target_dir=gtdir, index=index))
        jobdict[sample].append("mkdir {}".format(os.path.join(tgdir,sample+".genotype")))
        for chromosome in chromosomes:
            tiger_string = " && ".join(create_tiger_job(chromosome=chromosome,sample_name=sample, target_dir=tgdir,genomesize=gsize, gt_dir=gtdir,winsize=winsize))
            jobdict[sample].append(tiger_string)

In [19]:
#a is empty so we can ignore this
for sample in a.keys():
    if not sample in jobdict.keys():
        jobdict[sample] = []
        jobdict[sample].append("mkdir {}".format(os.path.join(tgdir,sample+".genotype")))
        for chromosome in chromosomes:
            tiger_string = " && ".join(create_tiger_job(chromosome=chromosome,sample_name=sample, target_dir=tgdir,genomesize=gsize, gt_dir=gtdir,winsize=winsize))
            jobdict[sample].append(tiger_string)

In [21]:
jobdict = {}
dict24 = []
shortjobs = {}
for sample, item in b.items():
    if not sample in jobdict.keys():
        #jobdict[sample] = []
        for windowsize, item2 in item.items():
            if len(item2)>8:
                jobdict[sample] = []
                for chromosome in item2:
                    if not chromosome == "24":
                        jobdict[sample].append(" && ".join(create_tiger_job(chromosome=chromosome,sample_name=sample, target_dir=tgdir,genomesize=gsize, gt_dir=gtdir,winsize=winsize)))
                    else :
                        dict24.append(" && ".join(create_tiger_job(chromosome=chromosome,sample_name=sample, target_dir=tgdir,genomesize=gsize, gt_dir=gtdir,winsize=winsize)))
            else:
                print(len(item2))
                shortjobs[sample] = []
                for chromosome in item2:
                    if not chromosome == "24":
                        shortjobs[sample].append(" && ".join(create_tiger_job(chromosome=chromosome,sample_name=sample, target_dir=tgdir,genomesize=gsize, gt_dir=gtdir,winsize=winsize)))
                    else :
                        dict24.append(" && ".join(create_tiger_job(chromosome=chromosome,sample_name=sample, target_dir=tgdir,genomesize=gsize, gt_dir=gtdir,winsize=winsize)))
                

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [88]:
#for k,i in jobdict.items():
    #print(len(i))

In [18]:
#dict24

In [23]:
scriptdir = '/home/tilman/storage3/process_stripes/stripes_optimise_F7/Stripes/stripes'
name = os.path.join(scriptdir, "{prefix}{sample}.job".format(prefix='all_chr_24_200', sample="mixed_samples"))
create_sbatch(joblines=dict24, name=name)

In [14]:
for k,i in shortjobs.items():
    print(k,len(i))

290315 0
297115 0
298215 0
238115 0
321615 0
302815 0
327215 0
286815 0
310915 0
311115 0
275215 0
280015 0
305915 0
288115 0
246715 0
333515 0
278915 0
308215 0
304015 0
325415 0
301515 0
302315 0
313315 0
305515 0
253815 0
271515 0
238415 0
260215 0
309215 0
251015 0
255215 1
273915 0
258015 0
256415 0
244815 0
255315 0
291615 0
248915 0
318715 0
325515 0
235315 0
257015 0
296015 0
295315 0
235015 0
283515 0
303715 0
315615 0
237815 0
283615 0
236315 0
260615 0
243115 4
307115 0
302415 0
320015 0
322315 0
271715 0
285515 0
291715 0
318215 0
294415 0
330415 0
253915 0
246815 0
331415 1
250315 0
329515 0
237615 0
301415 0
234215 1
325115 0
287315 0
240415 2
314815 0
280515 0
278115 0
311515 0
257815 0
296415 0
246515 0
329615 0
312515 0
295515 0
276815 0
273415 0
302615 0
293615 0
261915 0
302915 0
301115 0
249215 0
310715 0
312615 0
247415 0
319215 0
296215 0
241915 0
292815 0
298115 0
313615 1
306615 0
276215 0
308815 1
273715 0
294915 0
252415 0
286615 4
282715 0
311715 0
287915 0
2

In [15]:
def bundle_samples(jobdict, minsize=50, simplename=False):
    bundledict = {} #bundled jobdict
    bundle_name = ""
    current_jobs = []
    bundlecount = 1
    for sample, jobs in jobdict.items():
        if len(jobs)>0:
            #print(sample)
            current_jobs = current_jobs+jobs
            if simplename == True:
                bundle_name = 'bundle{}'.format(str(bundlecount))
            else:
                bundle_name = bundle_name+"_"+sample
            if len(current_jobs)>minsize:
                bundledict[bundle_name] = current_jobs
                bundle_name=""
                current_jobs=[]
                bundlecount = bundlecount+1
    bundledict[bundle_name] = current_jobs
    return bundledict


In [16]:
#bdict['_290315_297115_282615_298215_321615_302815_327215_286815_310915_311115_305915_288115_246715_333515_308215_304015_325415_301515_302315_313315_305515_271515_238415_260215_309215_251015_258015_256415_244815_291615_248915_318715_325515_235315_257015_296015_295315_235015_283515_303715_315615_237815_283615_236315_307115_302415_320015_322315_271715_285515_291715']

In [25]:
bdict = bundle_samples(jobdict=shortjobs, minsize=150, simplename=True)

In [16]:
bdict2= bundle_samples(jobdict=jobdict, minsize=150)

In [17]:
for key, item in bdict2.items():
    print( key, len(item))

 0


In [27]:
for key, item in bdict.items():
    print( key, len(item))

bundle1 131


In [28]:
create_batchfiles(jobdict=bdict2, scriptdir="/home/tilman/storage2/process_stripes/stripes_optimise_F15/Stripes/stripes", prefix="bundles_200421_200_002_")
create_batchfiles(jobdict=bdict, scriptdir="/home/tilman/storage2/process_stripes/stripes_optimise_F15/Stripes/stripes", prefix="bundles_200421_200_003_")

_244915_245515_243815_244515_244215_243715
172
_244715_244415_245415_245215_244315_244615
170
_245815_247815_245315
65
bundle1
131


In [52]:
#present_all